In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.7 MB/s eta 0:00:00


In [3]:
import torch
from transformers import MarianMTModel, MarianTokenizer, MarianConfig
import nltk

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
sub = 10000
train_tokenized = torch.load("/content/drive/MyDrive/Master_Project/Data/train_tokenized_{}.pth".format(sub)).to(device)
references = torch.load('/content/drive/MyDrive/Master_Project/Data/output_senetences_{}.pth'.format(sub))
ref_tokenized = tokenizer(references, return_tensors="pt", padding=True, truncation=True, add_special_tokens=True,max_length=206)


In [9]:
print(train_tokenized.attention_mask[:1])
print(ref_tokenized.attention_mask[:1])

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, src_data, tgt_data, src_attention_masks, tgt_attention_masks, tgt_labels):
        self.src_data = src_data
        self.tgt_data = tgt_data
        self.src_attention_masks = src_attention_masks
        self.tgt_attention_masks = tgt_attention_masks
        self.tgt_labels = tgt_labels

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        return (
            self.src_data[idx],
            self.src_attention_masks[idx].bool(),
            self.tgt_data[idx],
            self.tgt_attention_masks[idx].bool(),
            self.tgt_labels[idx]
        )

In [ ]:
torch.manual_seed(1)

train_dataset = MyDataset(train_tokenized.input_ids, ref_tokenized.input_ids, train_tokenized.attention_mask, ref_tokenized.attention_mask, train_tokenized.labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# train_x = TensorDataset(train_tokenized.input_ids, train_tokenized.attention_mask, train_tokenized.labels)
# train_x_dataloader = DataLoader(train_x, batch_size=32,drop_last=True)

# train_y = TensorDataset(ref_tokenized.input_ids, ref_tokenized.attention_mask)
# train_y_dataloader = DataLoader(train_y, batch_size=32,drop_last=True)

vocab = tokenizer.get_vocab()
len(vocab)

58101

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.functional import cross_entropy
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Seq2SeqTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_size=256, nhead=8, num_layers=3):
        super(Seq2SeqTransformer, self).__init__()
        self.embedding_src = nn.Embedding(src_vocab_size, emb_size)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size)
        self.transformer = Transformer(d_model=emb_size, nhead=nhead, num_encoder_layers=num_layers, num_decoder_layers=num_layers)
        self.fc = nn.Linear(emb_size, tgt_vocab_size)


    def forward(self, src_input_ids, src_attention_mask, tgt_input_ids, tgt_attention_mask):
        src_embedding = self.embedding_src(src_input_ids)
        src_embedding = self.pos_encoder(src_embedding)

        tgt_embedding = self.embedding_tgt(tgt_input_ids)
        tgt_embedding = self.pos_encoder(tgt_embedding)

        src_embedding = src_embedding.permute(1,0,2)
        tgt_embedding = tgt_embedding.permute(1,0,2)

        output = self.transformer(src_embedding, tgt_embedding, src_key_padding_mask=src_attention_mask, tgt_key_padding_mask=tgt_attention_mask)
        output = self.fc(output)
        return output

# Example usage
src_vocab_size = len(tokenizer.get_vocab())
tgt_vocab_size = len(tokenizer.get_vocab())
model = Seq2SeqTransformer(src_vocab_size, tgt_vocab_size).to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
     for batch_idx, (src_input_ids, src_attention_mask, tgt_input_ids, tgt_attention_mask, tgt_labels) in enumerate(train_loader):

        # tgt_labels = tgt_labels.to(device)
        src_input_ids = src_input_ids.to(device)
        src_attention_mask = src_attention_mask.to(device)
        tgt_input_ids = tgt_input_ids.to(device)
        tgt_attention_mask = tgt_attention_mask.to(device)

        optimizer.zero_grad()
        output = model(src_input_ids, src_attention_mask, tgt_input_ids, tgt_attention_mask)

        # selected_output = output[-1, :, :]
        # selected_target = tgt_labels[:, -1].to(device)

        # print(output.view(-1,output.size(-1)).shape)
        # print(tgt_input_ids.view(-1).shape)
        # print(output)
        # print(tgt_input_ids)

        tgt_input_ids = tgt_input_ids.long()

        loss = criterion(output.view(-1,output.size(-1)), tgt_input_ids.view(-1))
        loss.backward()

        optimizer.step()

        if batch_idx % 25 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')

torch.save(model.state_dict(), 'model_{}.pth'.format(num_epochs))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 0, Batch 0, Loss: 11.033472061157227
Epoch 0, Batch 25, Loss: 1.8903272151947021
Epoch 0, Batch 50, Loss: nan
Epoch 0, Batch 75, Loss: nan


KeyboardInterrupt: ignored